In [1]:
import numpy as np
import scipy.io.wavfile as wavfile
from scipy.signal import fftconvolve
from pydub import AudioSegment
import os

In [2]:
# Define paths to audio and RIR files

audio_file_paths = {
    "Trumpet": "audios\\01b_trumpet.wav",
    "Jazz": "audios\\23_jazz.wav",
    "violin": "audios\\02_violin.wav",
    "accordion": "audios\\20c_accordion.wav",
    "guitar": "audios\\11_guitar.wav"
}

rir_file_paths = {
    "small_office": "RIR\\h005_Office_Small_44txts.wav",
    "Auditorium": "RIR\\h252_Auditorium_1txts.wav",
    "Movie_Theater": "RIR\h115_MovieTheater_1txts.wav"
}

# Convertion to MONO

In [3]:
def normalize_audio(audio):
    max_amplitude = np.max(np.abs(audio))
    if max_amplitude == 0:
        return audio 
    normalized_audio = audio / max_amplitude
    return normalized_audio

In [4]:
for x in audio_file_paths:
    file_path = "audios\\" + x
    audio = AudioSegment.from_file(file_path)
    mono_audio = audio.set_channels(1)
    temp_file_path = "temp_mono.wav"
    mono_audio.export(temp_file_path, format="wav")
    sample_rate, mono_audio_np = wavfile.read(temp_file_path)
    mono_audio_np = mono_audio_np.astype(np.float32)
    normalized_audio_np = normalize_audio(mono_audio_np)
    wavfile.write(file_path, sample_rate, normalized_audio_np)
    os.remove(temp_file_path)
   
    print("Conversion to mono completed successfully!")

FileNotFoundError: [Errno 2] No such file or directory: 'audios\\Trumpet.wav'

In [9]:
def load_audio(file_path):
    sample_rate, data = wavfile.read(file_path)
    return sample_rate, data

In [10]:

def convolve_audio(audio, rir):
    if audio.ndim == 1:  # Mono
        return fftconvolve(audio, rir, mode='full')
    else:  # Stereo
        return np.array([fftconvolve(channel, rir, mode='full') for channel in audio.T]).T

In [11]:
rirs = {key: load_audio(path)[1] for key, path in rir_file_paths.items()}

for audio_key, audio_path in audio_file_paths.items():
    sample_rate, audio_data = load_audio(audio_path)
    for rir_key, rir_data in rirs.items():
        processed_audio = convolve_audio(audio_data, rir_data)
        processed_audio = np.int16(processed_audio / np.max(np.abs(processed_audio)) * 32767)
        output_path = f"output\\{audio_key}_{rir_key}.wav"
        wavfile.write(output_path, sample_rate, processed_audio)

C:\Users\abuma\AppData\Local\Temp\ipykernel_5804\843369094.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wavfile.read(file_path)


In [7]:
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import fftconvolve

def load_audio(file_path):
    sample_rate, data = wavfile.read(file_path)
    return sample_rate, data

def normalize_audio(audio):
    # Normalize the audio to the range -1 to 1
    return audio / np.max(np.abs(audio))

# Load RIR files
rirs = {key: load_audio(path)[1] for key, path in rir_file_paths.items()}

# Create output directory if it doesn't exist
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each audio file with each RIR
for audio_key, audio_path in audio_file_paths.items():
    sample_rate, audio_data = load_audio(audio_path)
    for rir_key, rir_data in rirs.items():
        processed_audio = convolve_audio(audio_data, rir_data)
        processed_audio = normalize_audio(processed_audio)
        processed_audio = processed_audio.astype(np.float32)  # Ensure the data is in float32 format
        output_path = os.path.join(output_dir, f"{audio_key}_{rir_key}.wav")
        wavfile.write(output_path, sample_rate, processed_audio)









C:\Users\abuma\AppData\Local\Temp\ipykernel_182140\3067238563.py:7: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, data = wavfile.read(file_path)
